<a href="https://colab.research.google.com/github/DhiaurrahmanRaziqRamadhan/MBKM-CNN/blob/main/Algoritma_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import csv
from google.colab import drive
from PIL import Image

In [ ]:
drive.mount('MBKM')

Drive already mounted at MBKM; to attempt to forcibly remount, call drive.mount("MBKM", force_remount=True).


In [ ]:
def processing_image(img_path):
  img = Image.open(img_path)

  new_size = (32, 32)
  resize_image = img.resize(new_size, Image.LANCZOS)
  matriks_rgb = resize_image.convert('RGB')

  # Konversi matriks RGB menjadi array numpy
  array_rgb = np.array(resize_image)

  # Cetak ukuran array_rgb
  print("Ukuran array_rgb:", array_rgb.shape)

  # Mendapatkan dimensi gambar (ukuran x, ukuran y, channel)
  ukuran_x, ukuran_y, channel = array_rgb.shape

  # Membuat array kosong untuk menyimpan nilai RGB per piksel
  nilai_rgb_per_piksel = np.zeros((ukuran_x, ukuran_y, channel), dtype=int)

  # Meloop melalui setiap piksel dan mendapatkan nilai RGB-nya
  for x in range(ukuran_x):
      for y in range(ukuran_y):
          for c in range(channel):
              nilai_rgb_per_piksel[x, y, c] = array_rgb[x, y, c]

  # Mendapatkan matriks Merah (Red)
  matriks_red = nilai_rgb_per_piksel[:, :, 0]

  # Mendapatkan matriks Hijau (Green)
  matriks_green = nilai_rgb_per_piksel[:, :, 1]

  # Mendapatkan matriks Biru (Blue)
  matriks_blue = nilai_rgb_per_piksel[:, :, 2]


  RGB_Values = '/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/RGB_Values'
  os.makedirs(RGB_Values, exist_ok=True)

  red_values = pd.DataFrame(matriks_red)
  red_values.to_csv('/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/RGB_Values/red_values.csv', index=False, header=False)

  green_values = pd.DataFrame(matriks_green)
  green_values.to_csv('/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/RGB_Values/green_values.csv', index=False, header=False)

  blue_values = pd.DataFrame(matriks_blue)
  blue_values.to_csv('/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/RGB_Values/blue_values.csv', index=False, header=False)

  # Simpan gambar yang telah diubah ukurannya
  resized_image_path = '/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/resized_image.png'
  resize_image.save(resized_image_path)

In [ ]:
train_folder = '/content/MBKM/MyDrive/MBKM/scenario2/train/1917051047'
img_name = 'face_1917051047_20221226_Pulang Cepat_2.png'

img_path = os.path.join(train_folder, img_name)
processing_image(img_path)

Ukuran array_rgb: (32, 32, 3)


# Perhitungan Algoritma CNN

## Perhitungan CNN setiap warna

In [ ]:
red_input = '/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/RGB_Values/red_values.csv'
green_input = '/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/RGB_Values/green_values.csv'
blue_input = '/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/RGB_Values/blue_values.csv'

red_output = 'red_output.csv'
green_output = 'green_output.csv'
blue_output = 'blue_output.csv'

In [ ]:
from scipy.signal import correlate2d

def apply_convolution(input_file, output_file, kernel):
  data = []
  with open(input_file, 'r') as file:
    read_csv = csv.reader(file)
    for row in read_csv:
      data.append(row)

  input_matriks = np.array(data, dtype=float)
  output_matriks = correlate2d(input_matriks, kernel, mode='valid')
  output_matriks = np.round(output_matriks).astype(int)

  return output_matriks

def saving_csv(matriks, output_file):
  with open(output_file, 'w', newline='') as file:
    write_csv = csv.writer(file)
    for row in matriks:
      write_csv.writerow(row)

In [ ]:
output_folder = '/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/output'
os.makedirs(output_folder, exist_ok=True)

kernel = np.array([
    [2, 1, 0, -1, -2],
    [2, 1, 0, -1, -2],
    [2, 1, 0, -1, -2],
    [2, 1, 0, -1, -2],
    [2, 1, 0, -1, -2]
])

# output matriks_red
output_matriks_red = apply_convolution(red_input, red_output, kernel)
output_path_red = os.path.join(output_folder, red_output)
saving_csv(output_matriks_red, output_path_red)

# output matriks_green
output_matriks_green = apply_convolution(green_input, green_output, kernel)
output_path_green = os.path.join(output_folder, green_output)
saving_csv(output_matriks_green, output_path_green)

# output matriks_blue
output_matriks_blue = apply_convolution(blue_input, blue_output, kernel)
output_path_blue = os.path.join(output_folder, blue_output)
saving_csv(output_matriks_blue, output_path_blue)

## Penjumlahan Matriks

In [ ]:
# Baca data dari file CSV menjadi array NumPy
red_values = np.genfromtxt('/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/output/red_output.csv', delimiter=',')
green_values = np.genfromtxt('/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/output/green_output.csv', delimiter=',')
blue_values = np.genfromtxt('/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/output/blue_output.csv', delimiter=',')

sum_matrix = red_values + green_values + blue_values

# Simpan hasil penjumlahan ke dalam file CSV
np.savetxt('/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/output/RGB_sum_matrix.csv', sum_matrix, delimiter=',', fmt='%d')

## Activation ReLU

In [ ]:
# Lakukan perhitungan ReLU pada matriks hasil penjumlahan
relu_matrix = np.maximum(sum_matrix, 0)

# Simpan hasil perhitungan ReLU ke dalam file CSV
np.savetxt('/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/output/relu_matrix.csv', relu_matrix, delimiter=',', fmt='%d')

## Pooling Layer

In [ ]:
def apply_max_pooling(input_matrix, pool_size):
  input_shape = input_matrix.shape
  pooled_rows = input_shape[0] // pool_size[0]
  pooled_cols = input_shape[1] // pool_size[1]

  pooled_matrix = np.zeros((pooled_rows, pooled_cols), dtype=int)

  for i in range(pooled_rows):
      for j in range(pooled_cols):
          start_row = i * pool_size[0]
          start_col = j * pool_size[1]
          pooled_value = np.max(input_matrix[start_row:start_row+pool_size[0], start_col:start_col+pool_size[1]])
          pooled_matrix[i, j] = pooled_value

  return pooled_matrix

# Ukuran pooling yang akan digunakan (misalnya 4x4)
pool_size = (4, 4)

# Lakukan Max Pooling pada matriks hasil Activation ReLU
pooled_matrix = apply_max_pooling(relu_matrix, pool_size)

# Simpan hasil Max Pooling ke dalam file CSV
np.savetxt('/content/MBKM/MyDrive/MBKM/AlgoritmaCNN/output/pooled_matrix.csv', pooled_matrix, delimiter=',', fmt='%d')

## Dropout Layer

In [ ]:
# Fungsi untuk melakukan dropout pada matriks
def apply_dropout(input_matrix, dropout_rate):
    mask = np.random.rand(*input_matrix.shape) < dropout_rate
    dropout_matrix = input_matrix * mask
    return dropout_matrix

# Load pooled matrix from CSV
pooled_output_filename = 'pooled_matrix.csv'
pooled_output_path = os.path.join(output_folder, pooled_output_filename)
pooled_matrix = np.genfromtxt(pooled_output_path, delimiter=',')

# Dropout parameters
dropout_rate = 0.5  # Misalnya, menggunakan dropout rate 0.5

# Apply dropout
dropout_matrix = apply_dropout(pooled_matrix, dropout_rate)

# Save dropout_matrix to CSV
dropout_output_filename = 'dropout_matrix.csv'
dropout_output_path = os.path.join(output_folder, dropout_output_filename)
np.savetxt(dropout_output_path, dropout_matrix, delimiter=',', fmt='%d')

## Flatten Layer

In [ ]:
# Flatten function
def flatten_matrix(input_matrix):
    return input_matrix.flatten()

# Flatten the pooled matrix
flattened_matrix = flatten_matrix(pooled_matrix)

# Save flattened_matrix to CSV
flattened_output_filename = 'flattened_matrix.csv'
flattened_output_path = os.path.join(output_folder, flattened_output_filename)
np.savetxt(flattened_output_path, flattened_matrix, delimiter=',', fmt='%d')

## Fully Connected Layer

In [ ]:
def calculate_hidden_layer(layer_input, weights):
  res = np.zeros((len(weights), 1), dtype = np.float64)
  for i in range(len(weights)):
    res[i] = np.sum(layer_input * weights[i])
  return res

In [ ]:
flattened_input = []
with open(flattened_output_path, 'r') as file:
  read_csv = csv.reader(file)
  for row in read_csv:
    flattened_input.append(float(row[0]))
flattened_input = np.array(flattened_input)

In [ ]:
# first hidden layer
weight_j = np.array([
    [0.4],
    [0.3],
    [0.1],
    [0.2]
], dtype=np.float64)
value_j = calculate_hidden_layer(flattened_input, weight_j)
print("Value J (First Hidden Layer):")
print(value_j)

# Second hidden layer
weight_h = np.array([
    [0.5],
    [0.2],
    [0.4],
    [0.1]
], dtype=np.float64)
value_h = calculate_hidden_layer(value_j, weight_h)
print("Value H (Second Hidden Layer):")
print(value_h)

# Output layer
weight_o = np.array([
    [0.3],
    [0.4]
], dtype=np.float64)
value_o = calculate_hidden_layer(flattened_input, weight_o)
print("Value O (Output Layer):")
print(value_o)

Value J (First Hidden Layer):
[[35459.2]
 [26594.4]
 [ 8864.8]
 [17729.6]]
Value H (Second Hidden Layer):
[[44324. ]
 [17729.6]
 [35459.2]
 [ 8864.8]]
Value O (Output Layer):
[[26594.4]
 [35459.2]]


In [ ]:
def softmax(x):
  exp_x = np.exp(x - np.max(x))  # Menghindari numerik instability dengan mengurangkan nilai maksimum dari setiap elemen
  return exp_x / np.sum(exp_x)

softmax_output = softmax(value_o)
print("Softmax Output:")
print(softmax_output)

Softmax Output:
[[0.]
 [1.]]
